In [1]:
# Expiry, symbol and option info extracted from NSE
import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan
    
#******               Symbols list          ****
#________________________________________________

# expiry = ''  # First run

symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

 
def get_data(symbol):

    while_flag = True
    while_counter = 0
    expiry = ''

    df = pd.DataFrame()

    while while_flag:

        if expiry != '':   # Mot First Run
            try:
                expiry = expiry[while_counter]
            except IndexError:
                while_flag = False
                break

        url_base = "https://www.nseindia.com/marketinfo/companyTracker/mtOptionKeys.jsp?companySymbol="
        url_end = "&indexSymbol=NIFTY&series=EQ&instrument=OPTSTK&date="
        url = url_base+symbol+url_end+expiry
        page = requests.get(url)
        doc = lh.fromstring(page.content)

        the_path= "//form"   # Get the form content. There are two forms
        expiries = doc.xpath(the_path)[0].text_content() # 0 has the expiry date

        strip_chars = [ord('\n'), ord('\xa0'), ord('\t')] # characters to be stripped
        char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

        sym_exp = expiries.translate(char_table).split()
        # symbol = sym_exp[2].strip('()') # Remove round () from the 3rd element for symbol

        # Get the expiries only, from the table
        expiry = [sym_exp[k+i] 
         for k, v in enumerate(sym_exp) 
         if v == '--Select--' 
         for i in range(len(sym_exp) - k)][1:]

        # Convert to datetime
        expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

        tbl_path = "//tr"
        tr_elements = doc.xpath(tbl_path)

        # Option table is contained in rows of length 21
        opt_table = [t for t in tr_elements if len(t) == 21]
        # tbl_head1 = [i.text_content() for i in opt_table[0]] # First row is header

        # Recreate the header
        tbl_head = ['cQuote', 'cOI', 'cChgOI', 'cLTP', 'cNetChg', 'cVolume', 
                     'cBidQty', 'cBidPrice', 'cOfferPrice', 'cOfferQty', 'Strike', 
                     'pBidQty', 'pBidPrice', 'pOfferPrice', 'pOfferQty', 
                     'pVolume', 'pNetChg', 'pLTP', 'pOI', 'pChgOI', 'pQuote']

        # Extract the opt_table
        opt_list = [[n.text_content().strip() 
                     for m, n in enumerate(v)] 
                     for k, v in enumerate(opt_table[1:])]

        # Create the dataframe with Strike and Expiry
        data = pd.DataFrame(columns=tbl_head, data=opt_list).drop(['cQuote', 'pQuote'], 1)

        data = data.replace(',', '', regex=True) # Remove comma from numbers
        data = data.apply(pd.to_numeric, errors='coerce') # Convert to numeric
        data.insert(0, 'Expiry', expiry_dt[while_counter])
        data.insert(0, 'Symbol', symbol)

        # Rearrange columns
        cols_beginning = ['Symbol', 'Expiry', 'Strike']
        data = data[cols_beginning + [c for c in data if c not in cols_beginning]]
        while_counter = while_counter + 1    
        df = df.append(data)
        
    return df

In [2]:
%%time
df_test = [get_data(symbol) for symbol in symbols[:1]]

Wall time: 200 ms


In [3]:
df_test

[   Symbol      Expiry  Strike      cOI   cChgOI    cLTP  cNetChg  cVolume  \
 0     ACC  2018-10-25  1160.0      NaN      NaN     NaN      NaN      NaN   
 1     ACC  2018-10-25  1180.0      NaN      NaN     NaN      NaN      NaN   
 2     ACC  2018-10-25  1200.0      NaN      NaN     NaN      NaN      NaN   
 3     ACC  2018-10-25  1220.0      NaN      NaN     NaN      NaN      NaN   
 4     ACC  2018-10-25  1240.0      NaN      NaN     NaN      NaN      NaN   
 5     ACC  2018-10-25  1260.0      NaN      NaN     NaN      NaN      NaN   
 6     ACC  2018-10-25  1280.0      NaN      NaN     NaN      NaN      NaN   
 7     ACC  2018-10-25  1300.0      NaN      NaN     NaN      NaN      NaN   
 8     ACC  2018-10-25  1320.0      NaN      NaN     NaN      NaN      NaN   
 9     ACC  2018-10-25  1340.0      NaN      NaN     NaN      NaN      NaN   
 10    ACC  2018-10-25  1360.0    400.0      NaN  198.00      NaN      NaN   
 11    ACC  2018-10-25  1380.0   1200.0      NaN  223.95      Na